In [1]:
from neat_src import loadHyp, updateHyp
from domain import load_task
from neat_src import DataGatherer, Neat 
from domain.config import games
from domain.task_gym import GymTask

game = games["slimevolley"]
task = GymTask(game)

hyp_default = 'p/default_neat.json'
hyp_adjust = "p/volley.json"
fileName = "volley"

hyp = loadHyp(pFileName=hyp_default, load_task=load_task)
updateHyp(hyp,load_task,hyp_adjust)

neat = Neat(hyp)

	*** Running with hyperparameters:  p/volley.json 	***


In [4]:
task.activations

array([1, 1, 1])

#### Convert Policy model to Ind in Neat 

In [1]:
import json
import numpy as np
import gym, os
import slimevolleygym
import slimevolleygym.mlp as mlp
from slimevolleygym.mlp import games as games
from slimevolleygym.mlp import Model
from slimevolleygym import multiagent_rollout as rollout


# Settings
random_seed = 612
population_size = 128
total_tournaments = 500000
save_freq = 1000


# Log results
logdir = "ga_rotation_je"
if not os.path.exists(logdir):
  os.makedirs(logdir)

def mutate(param, param_count):
  return param + np.random.normal(size=param_count) * 0.1

curr_policy = Model(games['slimevolleylite'])
prev_policy = Model(games['slimevolleylite'])

/var/folders/nn/nbsb8w3570zfgs23h2s0cdzm0000gn/T/ipykernel_73219/869475039.py:26: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  curr_policy = Model(games['slimevolleylite'])
/var/folders/nn/nbsb8w3570zfgs23h2s0cdzm0000gn/T/ipykernel_73219/869475039.py:27: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prev_policy = Model(games['slimevolleylite'])


In [3]:
# curr_policy.shapes
shapes = [(3, 2), (2, 1)]

In [15]:
# Slime Environment Specific model -> Ind convertion function 
def initIndiv(shapes): 
    nodes = [shapes[0][0]] + [s[0] for s in shapes[1:]] + [shapes[-1][-1]]

    nodeId = np.arange(0, sum(nodes))
    node = np.empty((3, len(nodeId)))
    node[0, :] = nodeId
    node[1, :] = 3 # Node types: [1:input, 2:output, 3:hidden, 4:bias]
    node[1, nodes[0]] = 1 # Input Nodes
    node[1, -nodes[-1]:] = 2 # Output Nodes
    node[2, :] = 1 # Activations 

    nConn = sum([t[0]*t[1] for t in curr_policy.shapes])
    conn = np.empty((5, nConn))
    conn[0, :] = np.arange(0, nConn)

    for i, (start, end) in enumerate(shapes): 
        minimal_id = np.sum([s[0]*s[1] for s in shapes[:i]]).astype(int)
        maximal_id = np.sum([s[0]*s[1] for s in shapes[:i+1]]).astype(int)
        conn[1, minimal_id:maximal_id] = np.tile(np.arange(0, start), end) + minimal_id
        conn[2, minimal_id:maximal_id] = np.repeat(np.arange(0, end), start) + minimal_id + start
        
        # Layer-wise Weight value 
        # conn[3, minimal_id:maximal_id] = np.random.randn(curr_policy.weight[i].flatten('F').shape)
        # conn[4, minimal_id:maximal_id] = 1

    conn[3, :] = np.random.randn(nConn)
    conn[4, :] = 1
    
    return node, conn 

node, conn = initIndiv(curr_policy.shapes)

In [12]:
start, end = shapes[0]
start, end

i = 0 
minimal_id = np.sum([s[0]*s[1] for s in shapes[:i]]).astype(int)
maximal_id = np.sum([s[0]*s[1] for s in shapes[:i+1]]).astype(int)
conn[1, minimal_id:maximal_id] = np.tile(np.arange(0, start), end) + minimal_id
conn[2, minimal_id:maximal_id] = np.repeat(np.arange(0, end), start) + minimal_id + start


In [21]:
from neat_src import Ind

indiv = Ind(conn=conn, node=node)

indiv.express()

# from neat_src import getNodeOrder
# order, wMat = getNodeOrder(node, conn) # Bug: claim that node & conn are not valid ... 


# individual needs to be converted back into policy model (for slime env use)

# indiv.mutAddConn(conn, node, None, 1)

True

In [27]:
# indiv.mutAddNode(indiv.conn, indiv.node, None, 1)
# indiv.mutAddConn(indiv.conn, indiv.node, None, 1)

In [3]:
nodeG = indiv.node
nIns = len(nodeG[0,nodeG[1,:] == 1]) + len(nodeG[0,nodeG[1,:] == 4])
nOuts = len(nodeG[0,nodeG[1,:] == 2])

In [4]:
nIns, nOuts

(13, 3)

In [5]:
indiv.mutAddConn(indiv.conn, indiv.node, None, 1, hyp)


(array([[ 0.   ,  1.   ,  2.   , ..., 36.   , 37.   , 38.   ],
        [ 0.   ,  1.   ,  2.   , ..., 10.   , 11.   , 12.   ],
        [13.   , 13.   , 13.   , ..., 15.   , 15.   , 15.   ],
        [-0.921,  0.323, -0.662, ...,  0.255,  1.317, -0.569],
        [ 1.   ,  1.   ,  1.   , ...,  1.   ,  1.   ,  1.   ]]),
 None)